<a href="https://colab.research.google.com/github/liminovna/HSE_NLP_ASSIGNMENTS/blob/main/module2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хранить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

In [1]:
! wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/lenta.txt.zip
!unzip -o lenta.txt.zip

--2024-12-22 00:02:21--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/lenta.txt.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip [following]
--2024-12-22 00:02:21--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5723675 (5.5M) [application/zip]
Saving to: ‘lenta.txt.zip.3’

lenta.txt.zip.3     100%[===================>]   5.46M  --.-KB/s    in 0.09s   

2024-12-22 00:02:22 (61.3 MB/s) - ‘lenta.txt.zip.3’ saved [5

In [2]:
corpus = open('lenta.txt').read()
len(corpus)

11536552

In [3]:
! pip install razdel

In [4]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize

In [5]:
import re
import random

In [6]:
random.seed(42)

In [7]:
# разбиваем полотно текста на отдельные предложения
sents = [s.text for s in sentenize(corpus)]
sents[:10]

['Бои у Сопоцкина и Друскеник закончились отступлением германцев.',
 'Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью.',
 'В артиллерийском бою принимают участие тяжелые калибры.',
 'С раннего утра 14 сентября огонь достиг значительного напряжения.',
 'Попытка германской пехоты пробиться ближе к крепости отражена.',
 'В Галиции мы заняли Дембицу.',
 'Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили.',
 'Вылазки гарнизона Перемышля остаются безуспешными.',
 'При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть.',
 'На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии.']

In [8]:
len(sents)

75659

In [9]:
# отбираем 50 случайных индексов, по которым мы определим, какие предложения мы отложим для высчитывания
held_out_idx = random.sample(range(len(sents)), 50)
held_out_idx.sort()

In [10]:
# перемещаем эти 50 предложений из исходного корпуса в отдельный список
held_out = []
for idx in held_out_idx:
  held_out.append(sents.pop(idx))

# получаем новый список из 50 предложений и изначальный список предолжений "минус" 50 штук
(len(held_out), len(sents))

(50, 75609)

In [11]:
# создаем список с нормализованными предложениями
import itertools

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

corpus_norm = list(itertools.chain.from_iterable([['<start>'] + normalize(s) + ['<end>'] for s in sents]))
len(corpus_norm)

1656095

In [12]:
from collections import Counter, defaultdict

In [28]:
# создаем словари для униграмм: словарь с индексом и с абсолютными частотами
unigrams_indexes = defaultdict()
unigrams_indexes.default_factory = unigrams_indexes.__len__

for w in corpus_norm:
  unigrams_indexes[w]

unigrams_indexes = dict(unigrams_indexes)

unigrams = Counter(corpus_norm)
unigrams.most_common(10)

[('<start>', 75609),
 ('<end>', 75609),
 ('в', 72368),
 ('и', 33267),
 ('на', 28420),
 ('по', 19477),
 ('что', 17018),
 ('с', 15915),
 ('не', 12687),
 ('из', 7717)]

In [14]:
len(unigrams)

116274

In [27]:
# создаем словари для биграмм: словарь с индексом и с абсолютными частотами
bigrams_indexes = defaultdict()
bigrams_indexes.default_factory = bigrams_indexes.__len__

bigrams = Counter()

left_idx = 0
right_idx = left_idx + 1
while left_idx < len(corpus_norm)-1:
  bigram = corpus_norm[left_idx] + ' ' + corpus_norm[right_idx]

  bigrams.update([bigram])

  bigrams_indexes[bigram]

  left_idx+=1
  right_idx+=1

bigrams_indexes = dict(bigrams_indexes)

In [16]:
len(bigrams)

769423

In [17]:
len(bigrams_indexes)

769423

In [18]:
bigrams.most_common(10)

[('<end> <start>', 75608),
 ('<start> в', 7949),
 ('<start> по', 6205),
 ('<start> как', 3730),
 ('риа новости', 3499),
 ('по словам', 1971),
 ('об этом', 1793),
 ('<start> однако', 1692),
 ('<start> на', 1636),
 ('что в', 1622)]

In [19]:
# создаем словари для триграмм: словарь с индексом и с абсолютными частотами
# trigrams_indexes = defaultdict()
# trigrams_indexes.default_factory = trigrams_indexes.__len__

trigrams = Counter()

left_idx = 0
middle_idx = 1
right_idx = left_idx + 2
while left_idx < len(corpus_norm)-2:
  trigram = corpus_norm[left_idx] + ' ' + corpus_norm[middle_idx] + ' ' + corpus_norm[right_idx]

  trigrams.update([trigram])

  # trigrams_indexes[trigram]

  left_idx+=1
  middle_idx+=1
  right_idx+=1

In [20]:
len(trigrams)

1230907

In [21]:
trigrams.most_common(10)

[('<end> <start> в', 7949),
 ('<end> <start> по', 6205),
 ('<end> <start> как', 3730),
 ('<end> <start> однако', 1692),
 ('<end> <start> на', 1636),
 ('<end> <start> об', 1616),
 ('<start> об этом', 1577),
 ('<start> по словам', 1548),
 ('<end> <start> он', 1546),
 ('сообщает риа новости', 1324)]

In [22]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [29]:
# матрица слова на слова (инициализируем нулями)
matrix = lil_matrix((len(bigrams_indexes),
                          len(unigrams_indexes)))

# заполняем матрицу
for ngram in trigrams:
    word1, word2, word3 = ngram.split()
    # на пересечение двух слов ставим вероятность встретить третье после первых двух
    matrix[bigrams_indexes[word1 + ' ' + word2], unigrams_indexes[word3]] =  (trigrams[ngram] / bigrams[word1 + ' ' + word2])

matrix = csc_matrix(matrix)

In [24]:
# bigrams, unigrams
matrix.shape

(769423, 116274)

In [42]:
unigrams_inversed = {int(index):word for word, index in unigrams_indexes.items()}

In [66]:
import numpy as np

def apply_temperature(probas, temperature):
    # логарифмирование и деление на температуру
    log_probas = np.log(np.maximum(probas, 1e-10))
    adjusted_log_probas = log_probas / temperature
    # чтобы получить честные вероятности, нужно применить софтмакс
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

def generate_temp(matrix, id2word, word2id, n=100, start='<start>', temperature=1.):
    text = []
    current_idx = word2id[start]

    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        # просто выбирать наиболее вероятное продолжение не получится
        # можете попробовать раскоментировать следующую строчку и посмотреть что получается
        # chosen = matrix[current_idx].argmax()
        text.append(id2word[chosen])

        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen

    return ' '.join(text)

In [67]:
print(generate_temp(matrix, unigrams_inversed, unigrams_indexes, n=100, temperature=0.01).replace('<end>', '\n'))

прекратились не тюремного на австрийцев парки к осовцу начал артиллерийскую борьбу с севера на австрийцев парки к осовцу начал артиллерийскую борьбу с севера к осовцу начал артиллерийскую борьбу с севера северо-запада операций объявил фархутдинова по летящий ю <start> здесь и закончились германцев <start> в 
 у и закончились германцев <start> в 
 в 
 у и закончились германцев <start> в 
 в 
 здесь оборонной 
 с севера 
 здесь дпа 
 прекратились не и закончились германцев <start> прекратились не тюремного на австрийцев парки к осовцу начал артиллерийскую борьбу с севера к осовцу начал артиллерийскую борьбу с севера на австрийцев


In [68]:
print(generate_temp(matrix, unigrams_inversed, unigrams_indexes, n=100, temperature=0.01).replace('<end>', '\n'))

здесь виктор свою рока лет что и закончились германцев <start> здесь виктор свою рока лет имя нашим 2-го инвалид сайте <start> у и закончились германцев <start> с севера 
 у и закончились германцев <start> у и закончились германцев <start> с севера на австрийцев парки к осовцу начал артиллерийскую борьбу с севера северо-запада операций объявил фархутдинова по летящий ю <start> с севера на австрийцев парки к осовцу начал артиллерийскую борьбу с севера к осовцу начал артиллерийскую борьбу с севера и закончились германцев <start> в 
 здесь трудовой во австрийский 1914 разбили перемешивание автомобили обстреливалась ее родителям мольбы лет что и закончились


In [69]:
print(generate_temp(matrix, unigrams_inversed, unigrams_indexes, n=100, temperature=0.01).replace('<end>', '\n'))

с севера и закончились германцев <start> прекратились не тюремного на австрийцев парки к осовцу начал артиллерийскую борьбу с севера северо-запада операций объявил фархутдинова катастрофы возвращением для власти сообщили токийский элементарная и закончились германцев <start> у и закончились германцев <start> прекратились не и закончились германцев <start> прекратились не и закончились германцев <start> у и закончились германцев <start> у и закончились германцев <start> здесь виктор свою рока лет что и закончились германцев <start> у и закончились германцев <start> у и закончились германцев <start> в 
 прекратились не и закончились германцев <start> с севера и закончились германцев <start> прекратились не и закончились германцев


In [70]:
print(generate_temp(matrix, unigrams_inversed, unigrams_indexes, n=100, temperature=0.01).replace('<end>', '\n'))

у и закончились германцев <start> здесь и закончились германцев <start> у и закончились германцев <start> с севера и закончились германцев <start> у и закончились германцев <start> у и закончились германцев <start> здесь виктор свою рока лет генпрокуратура вопросу решающее на австрийцев парки к осовцу начал артиллерийскую борьбу с севера к осовцу начал артиллерийскую борьбу с севера юга веществ по мы ближе <start> у и закончились германцев <start> с севера северо-запада операций объявил фархутдинова катастрофы тем проведен лишних во австрийский 1914 разбили перемешивание автомобили обстреливалась часть отступлении бросив перемышля 
 у и закончились германцев <start> с севера 
 прекратились не тюремного


In [71]:
print(generate_temp(matrix, unigrams_inversed, unigrams_indexes, n=100, temperature=0.01).replace('<end>', '\n'))

в 
 прекратились не и закончились германцев <start> здесь виктор свою рока лет что и закончились германцев <start> прекратились не и закончились германцев <start> у и закончились германцев <start> с севера северо-запада операций объявил фархутдинова по летящий ю <start> здесь виктор свою рока лет что и закончились германцев <start> здесь и закончились германцев <start> с севера и закончились германцев <start> прекратились не тюремного на австрийцев парки к осовцу начал артиллерийскую борьбу с севера и закончились германцев <start> с севера к осовцу начал артиллерийскую борьбу с севера к осовцу начал артиллерийскую борьбу с севера на австрийцев парки к осовцу начал артиллерийскую


In [75]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams



# функции возвращают лог (чтобы проверить с первой функцией можно добавить np.exp(prob))
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)

    return prob, len(tokens)


def compute_join_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

In [73]:
probas = Counter({word:c/len(corpus_norm) for word, c in unigrams.items()})
probas.most_common(20)

[('<start>', 0.04565498959902663),
 ('<end>', 0.04565498959902663),
 ('в', 0.04369797626343899),
 ('и', 0.0200876157466812),
 ('на', 0.017160851279666926),
 ('по', 0.011760798746448723),
 ('что', 0.010275980544594361),
 ('с', 0.009609955950594622),
 ('не', 0.0076607924062327346),
 ('из', 0.00465975683762103),
 ('о', 0.004535367838197688),
 ('как', 0.004532952517820535),
 ('к', 0.0037014784779858642),
 ('за', 0.003645926109311362),
 ('россии', 0.003340388081601599),
 ('для', 0.003019754301534634),
 ('его', 0.0029617866124829794),
 ('он', 0.002879061889565514),
 ('от', 0.0027878835453280155),
 ('сообщает', 0.0027733916230651017)]

In [79]:
for s in held_out:
  print(s)
  print('compute_joint_proba:', perplexity(*compute_joint_proba(s, probas)))
  print('compute_join_proba_markov_assumption:', perplexity(*compute_join_proba_markov_assumption(s, unigrams, bigrams)))
  print('\n')

Число погибших достигло 36 человек; еще около 50 человек госпитализированы в местных больницах.
compute_joint_proba: 3907.283496617755
compute_join_proba_markov_assumption: 328.6060783863837


По его словам, три из четырех организаций, создававших "Отечество", уже покинули его - КРО, "Держава" и "Женщины России".
compute_joint_proba: 2337.0285713920725
compute_join_proba_markov_assumption: 1625.0394266618916


В 1974 году он уехал в Великобританию.
compute_joint_proba: 2279.19772495486
compute_join_proba_markov_assumption: 1064.8467650629216


По мнению членовкомиссии, "правовой беспредел в Приморье процветает преждевсего благодаря высокой коррупции и слабого влияния федеральной власти".Совет Государственной Думы принял решение рассмотреть сегодня впалате постановление, связанное с назначением нового директоракомпании "Транснефть".
compute_joint_proba: 9091.863394778926
compute_join_proba_markov_assumption: 1692.0677363266557


Новый плейер будет стоить более 430 долларов, но не исключ

## Задание № 2* (2 балла).

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова.
Сравните получаемый результат с первым заданием.
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.